In [1]:
import asyncio
import nest_asyncio
import threading
import queue
import cv2
from concurrent.futures import ThreadPoolExecutor
# from ipywidgets import Image, Button, VBox
from IPython.display import display

nest_asyncio.apply()

In [2]:
import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt

In [3]:
openapi = ot2_api.OpentronsAPI()

In [4]:
r = openapi.get_run_info()

Total number of runs: 20
Current run ID: 72f37ea2-e2bc-4649-a531-4c4f08527032
Current run status: idle


In [10]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


<Response [200]>

In [5]:
openapi.create_run(verbose=True)

Request status:
<Response [201]>
{
  "data": {
    "id": "72f37ea2-e2bc-4649-a531-4c4f08527032",
    "ok": true,
    "createdAt": "2024-11-02T10:28:12.786305+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "hasEverEnteredErrorRecovery": false,
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": []
  }
}


<Response [201]>

In [6]:
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "2c6ce723-4f86-4307-9d21-cac9206933e8",
    "createdAt": "2024-11-02T10:28:16.634810+00:00",
    "commandType": "loadPipette",
    "key": "2c6ce723-4f86-4307-9d21-cac9206933e8",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "8d47b7a1-1ecc-4346-aca3-26ff43e07a85"
    },
    "startedAt": "2024-11-02T10:28:16.637597+00:00",
    "completedAt": "2024-11-02T10:28:18.649852+00:00",
    "intent": "setup",
    "notes": []
  }
}


<Response [201]>

In [12]:
openapi.move_to_coordinates((155,47,100), min_z_height=1, verbose=False)

<Response [201]>

In [5]:
manual_movement = utils.ManualRobotMovement(openapi)

In [7]:
keyboard.unhook_all()

In [6]:
import cv2
import asyncio
from concurrent.futures import ThreadPoolExecutor

# Function to display video using OpenCV in a separate thread
def show_frame(cap, window_initialized):
    # Only set window settings once (on the first frame)
    if not window_initialized:
        cv2.namedWindow("Video Stream", cv2.WINDOW_NORMAL)  # Allow resizing
        cv2.resizeWindow("Video Stream", 1348, 1011)  # Resize the window to 640x480
        window_initialized = True  # Mark window initialization complete
    
    ret, frame = cap.read()
    if ret:
        cv2.imshow("Video Stream", frame)
        # Wait for 1ms to check if 'q' is pressed to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            return False, window_initialized
    return True, window_initialized

async def stream_video():
    cap = cv2.VideoCapture(0)  # Open the camera
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2592)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1944)
    cap.set(cv2.CAP_PROP_FPS, 60)
    window_initialized = False  # Flag to track if window settings have been applied
    
    # Create a ThreadPoolExecutor to handle video display
    with ThreadPoolExecutor(max_workers=1) as executor:
        loop = asyncio.get_event_loop()
        
        while True:
            # Run the blocking show_frame function in a separate thread
            result, window_initialized = await loop.run_in_executor(executor, show_frame, cap, window_initialized)
            if not result:
                break  # Stop if 'q' is pressed or an error occurs

            # Use asyncio.sleep to avoid blocking the event loop
            await asyncio.sleep(0.03)  # ~30 FPS

    cap.release()
    cv2.destroyAllWindows()

# Start video streaming in the background with asyncio.create_task
async def start_video_stream():
    task = asyncio.create_task(stream_video())
    return task

# Run the background task asynchronously
task = await start_video_stream()

# Continue with other tasks in the event loop
print("Video stream is running in the background. You can continue with other tasks.")

Video stream is running in the background. You can continue with other tasks.


In [2]:
thing = OpentronsAPI()

In [3]:
thing.ENDPOINTS

{'runs': '/runs',
 'lights': '/robot/lights',
 'home': '/robot/home',
 'protocols': '/protocols',
 'commands': '/commands'}